In [1]:
from SBM_SDE import *
from obs_and_flow_classes_and_functions import *
import seaborn as sns
import torch
from torch import nn
import torch.distributions as d
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import math
from tqdm import tqdm
import random
from torch.autograd import Function
import argparse
import os
import sys
from pathlib import Path
import shutil
import pandas as pd

In [2]:
torch.manual_seed(0)
devi = torch.device("".join(["cuda:",f'{cuda_id}']) if torch.cuda.is_available() else "cpu")

cuda_id = 1
dt = .2 #SDE discretization timestep.
t = 1000 #Simulation run for T hours.
n = int(t / dt) 
t_span = np.linspace(0, t, n + 1)
t_span_tensor = torch.reshape(torch.Tensor(t_span), [1, n + 1, 1]) #T_span needs to be converted to tensor object. Additionally, facilitates conversion of I_S and I_D to tensor objects.
l_r = 1e-3
niter = 5000
piter = 1500
batch_size = 1 #Number of sets of observation outputs to sample per set of parameters.
state_dim_SCON = 3 #Not including CO2 in STATE_DIM, because CO2 is an observation.
state_dim_SAWB = 4 #Not including CO2 in STATE_DIM, because CO2 is an observation.

In [3]:
temp_ref = 283

#System parameters from deterministic model
u_M = 0.002
a_SD = 0.33
a_DS = 0.33
a_M = 0.33
a_MSC = 0.5
k_S_ref = 0.000025
k_D_ref = 0.005
k_M_ref = 0.0002
Ea_S = 75
Ea_D = 50
Ea_M = 50

#Diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
c_CO2 = 0.0001

SCON_C_params_dict = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC}

In [4]:
#Obtain SOC and DOC pool litter inputs for all SBMs.
i_s_tensor = 0.001 + 0.0005 * torch.sin((2 * np.pi / (24 * 365)) * t_span_tensor) #Exogenous SOC input function
i_d_tensor = 0.0001 + 0.00005 * torch.sin((2 * np.pi / (24 * 365)) * t_span_tensor) #Exogenous DOC input function

In [5]:
#Read-in deterministic data observations for use in inference.
obs_times, obs_means_CON, obs_error_CON = csv_to_obs_df('CON_synthetic_sol_df.csv', 4, 1000, 0.1)
obs_times, obs_means_AWB, obs_error_AWB = csv_to_obs_df('AWB_synthetic_sol_df.csv', 5, 1000, 0.1)
obs_times, obs_means_AWB_ECA, obs_error_AWB_ECA = csv_to_obs_df('AWB_ECA_synthetic_sol_df.csv', 5, 1000, 0.1)

In [6]:
def neg_log_lik(C_PATH, T_SPAN_TENSOR, DT, I_S_TENSOR, I_D_TENSOR, DRIFT_DIFFUSION, PARAMS_DICT, TEMP_REF):
    drift, diffusion_sqrt = DRIFT_DIFFUSION(C_PATH[:, :-1, :], T_SPAN_TENSOR[:, :-1, :], I_S_TENSOR[:, :-1, :], I_D_TENSOR[:, :-1, :], PARAMS_DICT, TEMP_REF)
    euler_maruyama_state_sample_object = d.multivariate_normal.MultivariateNormal(loc = C_PATH[:, :-1, :] + drift * DT, scale_tril = diffusion_sqrt * math.sqrt(DT))
    return -euler_maruyama_state_sample_object.log_prob(C_PATH[:, 1:, :]).sum(-1)

In [7]:
obs_model_CON_noCO2 = ObsModel(DEVICE = devi, TIMES = obs_times, DT = dt, MU = obs_means_CON[:-1, :], SCALE = obs_error_CON[:, :-1])
obs_model_AWB_noCO2 = ObsModel(DEVICE = devi, TIMES = obs_times, DT = dt, MU = obs_means_AWB[:-1, :], SCALE = obs_error_AWB[:, :-1])
obs_model_AWB_ECA_noCO2 = ObsModel(DEVICE = devi, TIMES = obs_times, DT = dt, MU = obs_means_AWB_ECA[:-1, :], SCALE = obs_error_AWB_ECA[:, :-1])

In [8]:
def train(DEVICE, L_R, NITER, PRETRAIN_ITER, BATCH_SIZE, OBS_MODEL, STATE_DIM, T, DT, N, T_SPAN_TENSOR, I_S_TENSOR, I_D_TENSOR, DRIFT_DIFFUSION, PARAMS_DICT, ANALYTICAL_STEADY_STATE_INIT):
    net = SDEFlow(DEVICE, BATCH_SIZE, OBS_MODEL, STATE_DIM, T, DT, N).to(DEVICE)
    optimizer = optim.Adam(net.parameters(), lr = L_R) 
    if PRETRAIN_ITER >= NITER:
        raise Exception("PRETRAIN_ITER must be < NITER.")
    best_loss_norm = 1e10
    best_loss_ELBO = 1e20
    norm_losses = [best_loss_norm] * 10
    ELBO_losses = [best_loss_ELBO] * 10
    C0 = ANALYTICAL_STEADY_STATE_INIT(I_S_TENSOR[0, 0, 0].item(), I_D_TENSOR[0, 0, 0].item(), PARAMS_DICT) #Calculate deterministic initial conditions.
    C0 = C0[(None,) * 2].repeat(BATCH_SIZE, 1, 1).to(DEVICE) #Assign initial conditions to C_PATH.
    with tqdm(total = NITER, desc = f'Train Diffusion', position = -1) as tq:
        for iter in range(NITER):
            net.train()
            optimizer.zero_grad()
            C_PATH, log_prob = net() #Obtain paths with solutions at times after t0.
            C_PATH = torch.cat([C0, C_PATH], 1) #Append deterministic CON initial conditions conditional on parameter values to C path. 
            if iter <= PRETRAIN_ITER:
                l1_norm_element = C_PATH - torch.mean(OBS_MODEL.mu, -1)
                l1_norm = torch.sum(torch.abs(l1_norm_element)).mean()
                best_loss_norm = l1_norm if l1_norm < best_loss_norm else best_loss_norm
                norm_losses.append(l1_norm.item())
                #l2_norm_element = C_PATH - torch.mean(OBS_MODEL.mu, -1)
                #l2_norm = torch.sqrt(torch.sum(torch.square(l2_norm_element))).mean()
                #best_loss_norm = l2_norm if l2_norm < best_loss_norm else best_loss_norm
                #l2_norm.backward()
                #norm_losses.append(l2_norm.item())
                if len(norm_losses) > 10:
                    norm_losses.pop(0)
                if iter % 10 == 0:
                    print(f"Moving average norm loss at {iter} iterations is: {sum(norm_losses) / len(norm_losses)}. Best norm loss value is: {best_loss_norm}.")
                    print('\nC_PATH mean =', C_PATH.mean(-2))
                    print('\nC_PATH =', C_PATH)
                l1_norm.backward()
            else:
                log_lik = neg_log_lik(C_PATH, T_SPAN_TENSOR.to(device), dt, I_S_TENSOR.to(device), I_D_TENSOR.to(device), DRIFT_DIFFUSION, PARAMS_DICT, TEMP_REF)
                ELBO = log_prob.mean() + log_lik.mean() - OBS_MODEL(C_PATH)
                best_loss_ELBO = ELBO if ELBO < best_loss_ELBO else best_loss_ELBO
                ELBO_losses.append(ELBO.item())
                if len(ELBO_losses) > 10:
                    ELBO_losses.pop(0)
                if iter % 10 == 0:
                    print(f"Moving average ELBO loss at {iter} iterations is: {sum(ELBO_losses) / len(ELBO_losses)}. Best ELBO loss value is: {best_loss_ELBO}.")
                    print('\nC_PATH mean =', C_PATH.mean(-2))
                    print('\n C_PATH =', C_PATH)
                ELBO.backward()
            torch.nn.utils.clip_grad_norm_(net.parameters(), 3.0)
            optimizer.step()
            if iter % 100000 == 0 and iter > 0:
                optimizer.param_groups[0]['lr'] *= 0.1
            tq.update()

In [9]:
train(devi, l_r, niter, piter, batch_size, obs_model_CON_noCO2, state_dim_SCON, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SCON_C, SCON_C_params_dict, analytical_steady_state_init_CON)


Train Diffusion:   0%|          | 0/5000 [00:00<?, ?it/s]

Moving average norm loss at 0 iterations is: 9000022788.976562. Best norm loss value is: 227889.765625.

C_PATH mean = tensor([[0.6117, 0.6914, 0.9139]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01],
         [8.2709e-01, 7.9033e-01, 8.8175e-01],
         [5.8314e-01, 6.9806e-01, 7.4934e-01],
         ...,
         [8.9601e-01, 7.9831e-01, 8.0336e-01],
         [8.9254e-01, 7.0059e-01, 2.4620e-01],
         [9.2251e-01, 5.9186e-01, 1.1602e-02]]], grad_fn=<CatBackward>)



Train Diffusion:   0%|          | 10/5000 [00:19<2:38:50,  1.91s/it]

Moving average norm loss at 10 iterations is: 218768.8578125. Best norm loss value is: 211444.0.

C_PATH mean = tensor([[3.4637, 0.2556, 0.4710]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[45.6603,  0.0715,  0.7147],
         [ 1.3127,  0.9626,  0.2885],
         [ 2.8074,  0.5820,  0.2604],
         ...,
         [ 1.9228,  0.2478,  0.7320],
         [ 1.8428,  0.2422,  0.3521],
         [ 1.9164,  0.3704,  0.2378]]], grad_fn=<CatBackward>)



Train Diffusion:   0%|          | 20/5000 [00:38<2:38:07,  1.91s/it]

Moving average norm loss at 20 iterations is: 202170.309375. Best norm loss value is: 194709.421875.

C_PATH mean = tensor([[6.8487, 0.1184, 0.2920]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[45.6603,  0.0715,  0.7147],
         [ 0.3499,  2.5412,  0.1739],
         [ 0.6134,  0.5675,  0.1693],
         ...,
         [ 7.2627,  0.1759,  0.3777],
         [ 6.5180,  0.1819,  0.2389],
         [ 6.8469,  0.1771,  0.0908]]], grad_fn=<CatBackward>)



Train Diffusion:   1%|          | 30/5000 [00:57<2:36:23,  1.89s/it]

Moving average norm loss at 30 iterations is: 184693.071875. Best norm loss value is: 176871.078125.

C_PATH mean = tensor([[10.5520,  0.0665,  0.1518]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01],
         [4.6941e-01, 3.7308e+00, 9.8634e-02],
         [2.3511e+00, 9.7190e-01, 8.9657e-02],
         ...,
         [1.0266e+01, 8.8101e-02, 2.0156e-01],
         [9.2303e+00, 9.0159e-02, 1.1625e-01],
         [1.0238e+01, 8.9022e-02, 3.1513e-02]]], grad_fn=<CatBackward>)



Train Diffusion:   1%|          | 34/5000 [01:07<2:45:06,  1.99s/it]


KeyboardInterrupt: 